In [ ]:
#colab
'''
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/aml_final/aml_final/
! git pull
! pip install setfit
'''
from datasets import load_dataset, Dataset
from setfit import SetFitModel, Trainer, TrainingArguments
import torch, gc

from data.dataset_config import DatasetConfig
from train.active_learning import ActiveTrainer, create_random_subset
from train.active_learning_config import ActiveLearningConfig
from train.reporter import Reporter
from train.metrics import camprehesive_metrics

In [ ]:
samples_per_cycle = 12
dataset_name = "dair-ai/emotion"
dataset = load_dataset(dataset_name)
dataset_config = DatasetConfig(text_column="text", num_classes=6)


train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

final_reporter = Reporter(dataset_name + "_final.csv", label_column=dataset_config.label_column)
cycle_reporter = Reporter(dataset_name + "_cycle.csv", report_train_args=False, label_column=dataset_config.label_column)
def after_train_callback(trainer: Trainer, dataset: Dataset, run_id: int):
    cycle_reporter.report(trainer=trainer, dataset=dataset, run_id=run_id)

In [ ]:
for dataset_seed in range(5):
    
